In [0]:
%%sh
aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=MONTH/date=2022-02-28/platform=1/

In [0]:

spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=MONTH/date=2021-12-31/platform=1/").createOrReplaceTempView("test")
spark.sql("select app_id, device_type, country, AU, count(*) from test group by app_id,  device_type, country, AU having count(*) > 1").show(20)


In [0]:

import pandas as pd
# spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=MONTH/date=2021-12-31/platform=1/").createOrReplaceTempView("test")
# spark.sql("select * from test where country='US' and app_id=20600000009072").show(20)
s3path = "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=MONTH/date=2021-12-31/platform=1/"
spark.read.parquet(s3path).createOrReplaceTempView("test")
df = spark.sql("select * from test where country='US' and app_id=20600000009072")
df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2022-03-02/mb_restore_part/", mode="append") #append

In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "daily": get_date_list(begin_date, end_date, "D"),
        "weekly": get_date_list(begin_date, end_date, "W-SAT"),
        "monthly": get_date_list(begin_date, end_date, "M")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-12-31", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2022-03-21", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
        try:
            s3path = "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type={range_type}/date={date}/platform=2/".format(range_type=range_type_mapping[granularity], date=date)
            spark.read.parquet(s3path).createOrReplaceTempView("test_basic_kpi")
            df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type, sum(AU) as sum from test_basic_kpi".format(date=date, granularity=granularity))
            df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2022-03-17/mb_restore_ios/", mode="append") #append
            print("PASS on {} {}".format(granularity, date))
        except Exception as e:
            print("ERROR on {} {}".format(granularity, date))

In [0]:

date='2022-03-17'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/mb_restore_ios/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/mb_restore_ios.csv".format(date),header = 'true')

In [0]:
%%sh
DATE='2022-03-17'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/mb_restore_ios.csv/ /tmp/$DATE/mb_restore_ios.csv/

echo "=======retention_week========="
cat /tmp/$DATE/mb_restore_ios.csv/*.csv

In [0]:

spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=WEEK/date=2022-03-12/platform=1/").createOrReplaceTempView("test")
spark.sql("select * from test where country='US' and app_id = 20600000009072").show(20)

In [0]:

s3path = "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=WEEK/date=2022-03-12/platform=1/"
spark.read.parquet(s3path).createOrReplaceTempView("test")
df = spark.sql("select * from test where country='US' and app_id = 20600000009072")
df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2022-03-03/mb_restore_part/", mode="append") #append

In [0]:

date='2022-03-03'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/mb_restore_part/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/mb_restore_part.csv".format(date),header = 'true')

In [0]:
%%sh
DATE='2022-03-03'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/mb_restore_part.csv/ /tmp/$DATE/mb_restore_part.csv/

echo "=======retention_week========="
cat /tmp/$DATE/mb_restore_part.csv/*.csv


In [0]:
%%sh
